In [1]:
import math
import json
import numpy
import pandas as pd
import networkx as nx
from itertools import combinations
from scipy.stats import pearsonr
from causality.inference.search import IC
from causality.inference.independence_tests import *

In [2]:
# load the data
df = pd.read_csv("data/wdvp_stats.tsv", 
                 sep="\t", 
                 header=0, 
                 skiprows=range(1, 5), 
                 index_col=0, 
                 thousands=',',
                 na_values=["-"])
df.drop("ISO Country code", axis=1, inplace=True)
df.dropna(axis=1, how="all", inplace=True)
df.fillna(df.mean(), inplace=True)

In [3]:
df.head()

,population,surface area (Km2),GINI index,happy planet index,human development index,world happiness report score,sustainable economic development assessment (SEDA),GDP (billions PPP),GDP per capita (PPP),GDP growth (annual %),...,regulatory quality,rule of law,control of corruption,judicial effectiveness score,government integrity score,property rights score,tax burden score,overall economic freedom score,financial freedom score,women MPs (% of all MPs)
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,36000000,652230,39.128378,20.20000,0.498,2.660000,51.883333,64.1000,1919.000000,1.500000,...,-1.3,-1.57,-1.52,28.200000,26.20000,17.900000,91.800000,51.300000,10.000000,27.7
Albania,2900000,27398,29.000000,36.80000,0.785,4.640000,53.100000,34.2000,11840.000000,2.600000,...,0.2,-0.40,-0.42,25.400000,39.90000,54.100000,85.100000,64.500000,70.000000,27.9
Algeria,41000000,2381740,35.300000,33.30000,0.754,5.250000,45.800000,612.5000,15027.000000,3.700000,...,-1.2,-0.86,-0.61,35.200000,29.00000,27.800000,74.000000,44.700000,30.000000,25.8
Andorra,77000,468,39.128378,26.49562,0.858,5.496232,51.883333,649.0375,20061.176796,2.414674,...,1.2,1.60,1.24,46.635519,41.86776,51.243169,76.603911,60.855618,48.379888,32.1
Angola,30000000,1246700,42.700000,26.49562,0.581,5.496232,28.400000,187.3000,6844.000000,3.000000,...,-1.0,-1.10,-1.41,25.400000,18.90000,36.000000,82.400000,48.600000,40.000000,38.2


In [4]:
# define the variable types: 'c' is 'continuous'
variables = [
    "GINI index",
    "happy planet index",
    "human development index",
    "world happiness report score",
    "sustainable economic development assessment (SEDA)",
    "GDP per capita (PPP)",
    "GDP growth (annual %)",
    "health expenditure  % of GDP",
    "health expenditure  per person",
    "education expenditure % of GDP",
    "education expenditure  per person ",
    "school life expectancy (YEARS)",
    "unemployment (%)",
    "government spending score",
    "government expenditure (% of GDP)",
    "political rights score ",
    "civil liberties score ",
    "political stability & absence of violence",
    "government effectiveness",
    "regulatory quality",
    "rule of law",
    "control of corruption",
    "judicial effectiveness score",
    "government integrity score",
    "property rights score",
    "tax burden score",
    "overall economic freedom score",
    "financial freedom score",
    "women MPs (% of all MPs)"
]

variable_types = {v: "c" for v in variables}

# run the search
ic_algorithm = IC(RobustRegressionTest)
causal_graph = ic_algorithm.search(df, variable_types)

In [5]:
# process and format the data for display
D = nx.DiGraph()
D.add_nodes_from(causal_graph.nodes())

causal_edges = {}
for n1, n2, data in causal_graph.edges(data=True):
    arrows = data["arrows"]
    if not arrows:
        causal_edges[(n1, n2)] = dict(genuine=data["marked"], directed=False, both_arrows=None)
    elif (n1 in arrows and n2 in arrows):
        causal_edges[(n2, n1)] = dict(genuine=data["marked"], directed=True, both_arrows=True)
    elif n1 in arrows:
        causal_edges[(n2, n1)] = dict(genuine=data["marked"], directed=True, both_arrows=False)
    elif n2 in arrows:
        causal_edges[(n1, n2)] = dict(genuine=data["marked"], directed=True, both_arrows=False)

def normalize(d):
    return (d - d.mean())/d.std()

for n1, n2 in combinations(D.nodes(), 2):
    r, pval = pearsonr(normalize(df[n1].values), normalize(df[n2].values))
    n1_n2 = causal_edges.get((n1, n2), None)
    n2_n1 = causal_edges.get((n2, n1), None)
    if not n1_n2 and not n2_n1: # no causal edge
        D.add_edge(n1, n2, **dict(genuine=None, directed=False, causal=False, both_arrows=None, r=r, pval=pval))
    elif n1_n2:
        D.add_edge(n1, n2, **{**n1_n2, **dict(causal=True, r=r, pval=pval)})
    elif n2_n1:
        D.add_edge(n2, n1, **{**n2_n1, **dict(causal=True, r=r, pval=pval)})

pos = nx.circular_layout(D)

In [8]:
nx.node_link_data(D)

{'directed': True,
 'multigraph': False,
 'graph': {},
 'nodes': [{'id': 'GINI index'},
  {'id': 'happy planet index'},
  {'id': 'human development index'},
  {'id': 'world happiness report score'},
  {'id': 'sustainable economic development assessment (SEDA)'},
  {'id': 'GDP per capita (PPP)'},
  {'id': 'GDP growth (annual %)'},
  {'id': 'health expenditure  % of GDP'},
  {'id': 'health expenditure  per person'},
  {'id': 'education expenditure % of GDP'},
  {'id': 'education expenditure  per person '},
  {'id': 'school life expectancy (YEARS)'},
  {'id': 'unemployment (%)'},
  {'id': 'government spending score'},
  {'id': 'government expenditure (% of GDP)'},
  {'id': 'political rights score '},
  {'id': 'civil liberties score '},
  {'id': 'political stability & absence of violence'},
  {'id': 'government effectiveness'},
  {'id': 'regulatory quality'},
  {'id': 'rule of law'},
  {'id': 'control of corruption'},
  {'id': 'judicial effectiveness score'},
  {'id': 'government integrity

In [7]:
pos

{'GINI index': array([1.0000000e+00, 7.4505806e-09]),
 'happy planet index': array([0.97662055, 0.21497045]),
 'human development index': array([0.90757543, 0.4198891 ]),
 'world happiness report score': array([0.79609311, 0.60517419]),
 'sustainable economic development assessment (SEDA)': array([0.64738631, 0.76216204]),
 'GDP per capita (PPP)': array([0.46840841, 0.88351209]),
 'GDP growth (annual %)': array([0.26752838, 0.96354998]),
 'health expenditure  % of GDP': array([0.05413896, 0.99853344]),
 'health expenditure  per person': array([-0.16178195,  0.98682655]),
 'education expenditure % of GDP': array([-0.37013814,  0.92897672]),
 'education expenditure  per person ': array([-0.56118715,  0.82768894]),
 'school life expectancy (YEARS)': array([-0.72599548,  0.68769944]),
 'unemployment (%)': array([-0.85685712,  0.51555396]),
 'government spending score': array([-0.94765318,  0.31930152]),
 'government expenditure (% of GDP)': array([-0.99413794,  0.10811912]),
 'political ri

In [16]:
# serialize and write
from networkx.readwrite.json_graph import node_link_data
import pickle

with open("data/graph", "wb") as f:
    f.write(pickle.dumps(node_link_data(D)))

with open("data/pos", "wb") as f:
    f.write(pickle.dumps(pos))